In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
## load the ann trained model , scaler pickle ,onehot
model = load_model('model.h5')

## load the encoder and scaler
with open('ohe_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scalar.pkl', 'rb') as file:
    scalar = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 800,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 60,
    'Tenure': 5,
    'Balance': 120000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 100000
}

In [4]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Sudeep\Documents\Complete GenAI\Project-1 ANN CLASSIFICATION\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,Male,60,5,120000,2,1,1,100000


In [6]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,1,60,5,120000,2,1,1,100000


In [7]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,60,5,120000,2,1,1,100000,1.0,0.0,0.0


In [8]:
input_scaled=scalar.transform(input_df)
input_scaled

array([[ 1.55925793,  0.90134069,  1.99158224, -0.00371267,  0.69556431,
         0.80268517,  0.64104192,  0.97067965,  0.01154461,  1.00125078,
        -0.57715782, -0.57850497]])

In [9]:
prediction = model.predict(input_scaled)

prediction_prob = prediction[0][0]
prediction_prob

1/1 [==============================] - 0s 91ms/step


0.11098332

In [10]:
if prediction_prob>0.5:
    print("Customer is likely to churn")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn
